In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\jacob\\Desktop\\Technica_Assesment\\Question_2\\Mushroom_Classification'

In [2]:
os.environ["MLFLOW_TRACKING_URI"]="https://github.com/Jakee4488/Technical_Assesment"
os.environ["MLFLOW_TRACKING_USERNAME"]="Jakee4488"

In [3]:

import dagshub
dagshub.init(repo_owner='Jakee4488', repo_name='Technical_Assesment', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Jakee4488

Initialized MLflow to track repo "Jakee4488/Technical_Assesment"

Repository Jakee4488/Technical_Assesment initialized!

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    test_data_target: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from Mushroom_Classification_src.constants import *
from Mushroom_Classification_src.utils.common import read_yaml, create_directories ,save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            test_data_target=config.test_data_target,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Jakee4488/Technical_Assesment.mlflow",
           
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path).values
        test_data_target=pd.read_csv(self.config.test_data_target).values

        model = joblib.load(self.config.model_path)


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_data)

            (rmse, mae, r2) = self.eval_metrics(test_data_target, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-27 23:51:11,232: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-27 23:51:11,244: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-27 23:51:11,274: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-27 23:51:11,282: INFO: common: created directory at: artifacts]
[2024-11-27 23:51:11,289: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-27 23:51:11,701: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\jacob\Desktop\Technica_Assesment\Question_2\Mushroom_Classification\Mushroom_Env\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2024/11/27 23:51:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
